In [ ]:
import pandas as pd
import numpy as np
import librosa 

import seaborn as sn
import matplotlib.pyplot as plt
import IPython.display as ipd #palying audio
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import os
import csv
import soundfile
import sklearn
from matplotlib.colors import Normalize
import glob
from scipy.stats import kurtosis, skew
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_validate
import pickle
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

In [ ]:
path = os.chdir('C:/Users/makye/OneDrive/Documents/Career/BeCode/study/Data_preprocessing/audioProcessing/pump')
print(os.getcwd())

In [ ]:
# load dataset

def loadData():
    
    #Pulling the data
    path = 'martin_pump40.csv'
    df = pd.read_csv(path)
    return df

df = loadData()
df


In [ ]:
# #Creating header for features
# header = ''

# variables = ['chroma_stft', 'mel_spec',  'rms_f',  'spec_cent', 'spec_bw', 'rolloff', 'zcrf']

# for i in variables:
    
#     header += f' mean_{i}'
#     header += f' min_{i}'
#     header += f' max_{i}'
#     header += f' std_{i}'
#     header += f' median_{i}'
    
# for i in range(1, 41):
#     header += f' mean_mfcc{i}'
        
# for i in range(1, 41):
#     header += f' min_mfcc{i}'
        
# for i in range(1, 41):
#     header += f' max_mfcc{i}'
        
# for i in range(1, 41):
#     header += f' std_mfcc{i}'
        
# for i in range(1, 41):
#     header += f' median_mfcc{i}'
    
# header = header.split()

In [ ]:
# # Adding header to features

# features  #Selecting features
# features.columns = header # Adding header to features
# features

In [9]:
#defining features and target/scaling

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

scaler = StandardScaler()
# keep our unscaled features just in case we need to process them alternatively
X_scaled = scaler.fit_transform(X)

scaler = MinMaxScaler()
# keep our unscaled features just in case we need to process them alternatively
X_minmax = scaler.fit_transform(X)


In [10]:
############# Unscaled test/train set #############
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

############ Standard Scaled test/train set ###########
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_scaled, X_test_scaled, _, _ = train_test_split(
    X_scaled, 
    y, 
    test_size=0.2, 
    random_state=42
)

############# MinMax Scaled test/train set ###############
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_minmax, X_test_minmax, _, _ = train_test_split(
    X_scaled, 
    y, 
    test_size=0.2, 
    random_state=42
)

In [ ]:
#Unscaled:

classification_models = [
    KNeighborsClassifier(),#(3),
    SVC(kernel='linear'),#, C=0.025),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),#max_depth=5),
    RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores = []
for model in classification_models:
    model.fit(X_train, y_train) #Unscaled train
    score = model.score(X_test, y_test) #unscaled test
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

In [ ]:
#Scaled
classification_models = [
    KNeighborsClassifier(),#(3),
    SVC(kernel='linear'),#, C=0.025),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),#max_depth=5),
    RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores = []
for model in classification_models:
    model.fit(X_train_scaled, y_train) #scaled train
    score = model.score(X_test_scaled, y_test) #Scaled test
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

In [11]:
#RandomForest Classifier (Scaled features)
classifier = RandomForestClassifier(random_state=42, n_estimators=150)
# classifier  = SVC()
classifier.fit(X_train_scaled, y_train) #scaled train
# score = model.score(X_test_scaled, y_test)
y_pred = classifier.predict(X_test_scaled)

In [ ]:
def Metrics(y_test, y_pred):
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print(accuracy_score(y_test, y_pred))
    
Metrics(y_test, y_pred)

In [ ]:
#Plotting confusion matrix
cm = confusion_matrix(y_test,y_pred)
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot = True)
plt.xlabel('Predicted')
plt.ylabel('True Values')

In [ ]:
#Cross Validation????
cv = cross_validate(classifier, X_scaled, y, cv=5)
print(cv['test_score'])
print(cv['test_score'].mean())

In [ ]:
#Save model
filename = 'finalized_model_pumpRF76Sc.sav'
pickle.dump(classifier, open(filename, 'wb'))
 

In [ ]:
# Calculate & plot Feature importance 
importance = list(classifier.feature_importances_)
importance.sort()

# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

# plot feature importance
plt.barh([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
# Calculating and plotting feature importance

def plot_feature_importance(importance,names,model_type):

                #Create arrays from feature importance and feature names
                feature_importance = np.array(importance)
                feature_names = np.array(names)

                #Create a DataFrame using a Dictionary
                data={'feature_names':feature_names,'feature_importance':feature_importance}
                fi_df = pd.DataFrame(data)

                #Sort the DataFrame in order decreasing feature importance
                fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

                #Define size of bar plot
                plt.figure(figsize=(10,8))
                #Plot Searborn bar chart
                sn.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
                #Add chart labels
                plt.tick_params(axis='x', which='major', labelsize= 'small', rotation=45)
                plt.title(model_type + 'FEATURE IMPORTANCE')
                plt.xlabel('FEATURE IMPORTANCE')
                plt.ylabel('FEATURE NAMES')


# calling function
plot_feature_importance(importance, features.columns,'RANDOM FOREST')


In [ ]:
# Feature selection by feature importance of random forest classifier

#Selecting important features
sel = SelectFromModel(RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1))
sel.fit(X_train, y_train)
sel.get_support()#Returns array of boolean values with importance status

In [ ]:
#Feature columns
X_train.columns

In [ ]:
#retained features
retained = X_train.columns[sel.get_support()]
retained

In [ ]:
#Number of retained features
len(retained)

In [ ]:
# overall mean importance of selected features
print(np.mean(sel.estimator_.feature_importances_))

#features above the mean importance carry more weight
sel.estimator_.feature_importances_

In [ ]:
#function to call alogarithm for RandomForest
def run_randomForest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=35, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print('Accuracy:', accuracy_score(y_test, y_pred))

In [ ]:
%%time
#Calling function on untrimmed dataset

run_randomForest(X_train, X_test, y_train, y_test)

In [ ]:
%%time

X_train_rfc = sel.transform(X_train)
X_test_rfc = sel.transform(X_test)

#Calling function on trimmed dataset (with selected features)
run_randomForest(X_train_rfc, X_test_rfc, y_train, y_test)



In [ ]:

X_train_rfc 

In [ ]:
#Random Forest unscaled, SVC unscaled

In [ ]:
#create overall dataframe 

#Append target to features dataframe
new_df = pd.concat([features, y], axis=1)

#Adding column 'machine ID' (pump = 4)
new_df['Machine_ID'] = 4
new_df


#CSV of dataFrame

def createCsv(df):
    

    df.to_csv('martin_pump40.csv', index=False)
    
createCsv(new_df)

In [ ]:

# some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded